In [42]:
import xmltodict
import matlab.engine
eng = matlab.engine.start_matlab()


In [43]:
def test_xml_to_dict_equivalence(file_path):
    # Prepare a sample XML file
    # xml_content = '''
    # <XMLname attrib1="Some value">
    #     <Element>Some text</Element>
    #     <DifferentElement attrib2="2">Some more text</DifferentElement>
    #     <DifferentElement attrib3="2" attrib4="1">Even more text</DifferentElement>
    # </XMLname>
    # '''
    # with open(file_path, 'w') as file:
    #     file.write(xml_content)

    # Convert XML to dictionary using xml_to_dict function
    dict1 = xml_to_dict(file_path)

    # Convert XML to dictionary using xml2struct2 function (implemented in MATLAB)
    # For comparison purposes, you may need to write an equivalent Python function
    # or convert the MATLAB function to Python
    dict2 = convert_xml_to_dict_using_matlab_function(file_path)

   # # Compare the resulting dictionary structures
   #  assert dict1 == dict2, "Dictionary structures from both functions are not equal"
    if dict1 == dict2:
        print("Unit test passed: Both functions produce equivalent dictionary structures")
    else:
        print ("Dictionary structures from both functions are not equal")

    return dict1,dict2

def convert_xml_to_dict_using_matlab_function(file_path):

    # Start the MATLAB Engine session
    eng = matlab.engine.start_matlab()
    #path to matlab fucntion
    eng.addpath(r"\\10.99.68.52\Kiemendata\Valentina Matos\Dashboard Project\CODA Hub\base")
    # Call the MATLAB function with the provided file path
    dict2 = eng.xml2struct2(file_path)
    # Close the MATLAB Engine session
    eng.quit()
    return dict2

#same funciton in python
def xml_to_dict(file_path):
    with open(file_path, 'r') as file:
        xml_data = file.read()
        xml_dict = xmltodict.parse(xml_data[1:], dict_constructor=dict)
    return xml_dict


In [46]:
# Run the unit test
file_path = 'SG_013_0061.xml'
dict_py,dict_mat=test_xml_to_dict_equivalence(file_path)
print('')
print('Python Dictionary')
print(dict_py)

print('')
print('Python Structure')
print(dict_mat)

ExpatError: syntax error: line 1, column 0

#Separate functions

In [24]:
# Start the MATLAB Engine session
eng = matlab.engine.start_matlab()
#path to matlab fucntion
eng.addpath(r"\\10.99.68.52\Kiemendata\Valentina Matos\Dashboard Project\CODA Hub\base")
# Call the MATLAB function with the provided file path
file_path = 'SG_013_0061.xml'
dict2 = eng.xml2struct2(file_path)
# Close the MATLAB Engine session
eng.quit()

In [25]:
dict2

{'Annotations': {'Annotation': [{'Attributes': {'Id': '1',
     'Incremental': '0',
     'LineColor': '65280',
     'LineColorReadOnly': '0',
     'MacroName': '',
     'MarkupImagePath': '',
     'Name': 'islets',
     'NameReadOnly': '0',
     'ReadOnly': '0',
     'Selected': '0',
     'Type': '4',
     'Visible': '1'},
    'Regions': {'RegionAttributeHeaders': {'AttributeHeader': [{'Text': '',
        'Attributes': {'ColumnWidth': '-1', 'Id': '9999', 'Name': 'Region'}},
       {'Text': '',
        'Attributes': {'ColumnWidth': '-1', 'Id': '9997', 'Name': 'Length'}},
       {'Text': '',
        'Attributes': {'ColumnWidth': '-1', 'Id': '9996', 'Name': 'Area'}},
       {'Text': '',
        'Attributes': {'ColumnWidth': '-1', 'Id': '9998', 'Name': 'Text'}},
       {'Text': '',
        'Attributes': {'ColumnWidth': '-1',
         'Id': '1',
         'Name': 'Description'}}]}},
    'Plots': {'Text': ''}},
   {'Attributes': {'Id': '2',
     'Incremental': '0',
     'LineColor': '65535',


In [37]:
#python structure

with open(file_path, 'r') as file:
        xml_data = file.read()
        xml_dict = xmltodict.parse(xml_data[1:], dict_constructor=dict)

ExpatError: syntax error: line 1, column 0

In [35]:
import xml.etree.ElementTree as ET
import os
import xml.dom.minidom
from org.apache.xerces.dom import DeferredDocumentImpl, DeferredElementImpl

#let's build it from scratch
def xml2struct_python(file):

    xDoc = file

    theNode = xDoc
    # Recurse over node children.
    children = {}
    ptext = {}
    textflag = 'Text'

    if theNode.hasChildNodes():
        childNodes = theNode.childNodes
        numChildNodes = len(childNodes)
        for count in range(numChildNodes):
        theChild = childNodes.item(count)
        text, name, attr, childs, textflag = getNodeData(theChild)


def parseChildNodes(theNode):
    '''Parse child nodes of an XML node.

    Args:
    theNode (xml.dom.Node): The XML node to parse.

    Returns:
    tuple: A tuple containing:
        - dict: Dictionary containing child nodes.
        - dict: Dictionary containing parent text.
        - str: Text flag indicating the type of text.

    Parses the child nodes of an XML node and returns information about the child nodes and parent text.
    '''

    # Recurse over node children.
    children = {}
    ptext = {}
    textflag = 'Text'

    if theNode.hasChildNodes():
        childNodes = theNode.childNodes
        numChildNodes = childNodes.length

        for count in range(numChildNodes):
            theChild = childNodes.item(count)
            text, name, attr, childs, textflag = getNodeData(theChild)

            if name != '#text' and name != '#comment' and name != '#cdata_dash_section':
                # XML allows the same elements to be defined multiple times,
                # put each in a different cell
                if name in children:
                    if not isinstance(children[name], list):
                        children[name] = [children[name]]
                    index = len(children[name]) + 1
                    children[name].append(childs)
                    if text:
                        children[name][index] = text
                    if attr:
                        children[name][index]['Attributes'] = attr
                else:
                    # Add previously unknown (new) element to the structure
                    children[name] = childs
                    if text and text:
                        children[name] = text
                    if attr:
                        children[name]['Attributes'] = attr
            else:
                ptextflag = 'Text'
                if name == '#cdata_dash_section':
                    ptextflag = 'CDATA'
                elif name == '#comment':
                    ptextflag = 'Comment'

                # This is the text in an element (i.e., the parentNode)
                if text[textflag].strip() != '':
                    if ptextflag not in ptext or not ptext[ptextflag]:
                        ptext[ptextflag] = text[textflag]
                    else:
                        # What to do when element data is as follows:
                        # <element>Text <!--Comment--> More text</element>
                        # Just append the text
                        ptext[ptextflag] += text[textflag]

    return children, ptext, textflag

def getNodeData(theNode):
    # Create structure of node info.

    # Make sure name is allowed as a structure name
    name = getNodeName(theNode)
    name = name.replace('-', '_dash_')
    name = name.replace(':', '_colon_')
    name = name.replace('.', '_dot_')
    attr = parseAttributes(theNode)
    if not attr:
        attr = {}

    # Parse child nodes
    childs, text, textflag = parseChildNodes(theNode)

    if not childs and not text:

        text[textflag] = getTextContent(theNode)

    return text, name, attr, childs, textflag

def parseAttributes(theNode):
    # Create attributes structure.
    attributes = {}

    if theNode.hasAttributes():
        theAttributes = theNode.attributes
        numAttributes = theAttributes.length

        for count in range(numAttributes):
            attrib = theAttributes.item(count)
            attr_name = attrib.name.replace('-', '_dash_')
            attr_name = attr_name.replace(':', '_colon_')
            attr_name = attr_name.replace('.', '_dot_')
            attr_value = attrib.value
            attributes[attr_name] = attr_value

    return attributes





# Example usage:
# s = xml2struct('SG_013_0061.xml')
# print(s)


{'Annotation': [{'Attributes': {'Id': '1', 'Name': 'islets', 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '65280', 'Visible': '1', 'Selected': '0', 'MarkupImagePath': '', 'MacroName': ''}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {'Id': '2', 'Name': 'normal duct', 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '65535', 'Visible': '1', 'Selected': '0', 'MarkupImagePath': '', 'MacroName': ''}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {'Id': '3', 'Name': 'blood vessel', 'ReadOnly': '0', 'NameReadOnly': '0', 'LineColorReadOnly': '0', 'Incremental': '0', 'Type': '4', 'LineColor': '255', 'Visible': '1', 'Selected': '0', 'MarkupImagePath': '', 'MacroName': ''}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {}}, {'Attributes': {'Id': '4', 'Name': 'fat', 'ReadOnly': '0', 'NameReadOnly': '0'

In [36]:
children = {}

